# Union / Concat Tables

The _concat_ function mimics the functionality of UNION in SQL queries and replaces the Merge functionality in Excel. It is a powerful function for dataframes in Pandas, that is mostly missing from Excel. 

For this example, we will transform a table with game scores into a calculated standing table.

[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/aiola-lab/from-excel-to-pandas/blob/master/notebooks/03.04_concat_tables.ipynb)

In [1]:
import pandas as pd

## Loading raw table data

For this example, we will take an example of score table from one of the sports in the US, and calculate the standing table of that league using the league rules for points and rankings. 



In [5]:
url = 'https://www.worldfootball.net/schedule/eng-premier-league-2020-2021-spieltag/1/'

import requests
from io import StringIO

response = requests.get(url)
response

<Response [200]>

In [7]:
scores = (
    pd
    .read_html(
        StringIO(
            response.text
        )
    )
)

As you can see in the [web site](https://www.worldfootball.net/schedule/eng-premier-league-2020-2021-spieltag/1/), the format of the table is simple, and we can parse it with:
* Take the second (index of 1) on the web site
* Keep only the columns 2, 4, 5
* Rename the columns to reflect the names of the teams and the score
* Parse the score with the first (index of 0) character is the number of goals of the home team
* and the third (index of 2) character is the number of goals of the away team

In [70]:
(
    scores[1]
    [[2,4,5]]
    .rename(columns={
        2: 'Home',
        4: 'Away',
        5: 'Score'
    })
    .assign(home_goals = lambda r: r.Score.str[0])
    .assign(away_goals = lambda r: r.Score.str[2])
)


,Home,Away,Score,home_goals,away_goals
0,Fulham FC,Arsenal FC,0:3 (0:1),0,3
1,Crystal Palace,Southampton FC,1:0 (1:0),1,0
2,Liverpool FC,Leeds United,4:3 (3:2),4,3
3,West Ham United,Newcastle United,0:2 (0:0),0,2
4,West Bromwich Albion,Leicester City,0:3 (0:0),0,3
5,Tottenham Hotspur,Everton FC,0:1 (0:0),0,1
6,Sheffield United,Wolverhampton Wanderers,0:2 (0:2),0,2
7,Brighton & Hove Albion,Chelsea FC,1:3 (0:1),1,3
8,Burnley FC,Manchester United,0:1 (0:0),0,1
9,Manchester City,Aston Villa,2:0 (0:0),2,0


Now we have the scores of the first round with the hosting team (`Home`), the visiting team (`Away`) and the number of goals that each team scored.  

## Concatinate multuple tables

We want to concatinate the scores of multiple rounds. Using a for-loop we will load the results of each round and add them to a list of rounds.

In [36]:
round_tables = []
for round in range(1,5):
    url = f'https://www.worldfootball.net/schedule/eng-premier-league-2020-2021-spieltag/{round}/'
    response = requests.get(url)
    round_table = (
        pd
        .read_html(
            StringIO(
                response.text
            )
        )
        [1]
        [[2,4,5]]
        .rename(columns={
            2: 'Home',
            4: 'Away',
            5: 'Score'
        })
        .assign(home_goals = lambda r: r.Score.str[0].astype(int))
        .assign(away_goals = lambda r: r.Score.str[2].astype(int))
    )
    round_tables.append(round_table)
    

## Concatinate the rounds scores together

In Excel it is not easy to stitch multiple tables together into a single larger table. Nevertheless, with Pandas it is easy, as we can see in the following example:
* Concatinate the list of round scores
* Calculate the number of points of the home team (3 for a win, 1 for a draw)
* Calculate the number of points of the away team

In [37]:
import numpy as np

scores_table = (
    pd
    .concat(round_tables)
    .assign(home_points = lambda r : np.select(
            [
                r.home_goals > r.away_goals, 
                r.home_goals == r.away_goals, 
            ], 
            [
                3, 
                1
            ], 
            default=0
        )
    )
        .assign(away_points = lambda r : np.select(
            [
                r.home_goals < r.away_goals, 
                r.home_goals == r.away_goals, 
            ], 
            [
                3, 
                1
            ], 
            default=0
        )
    )

)

scores_table

,Home,Away,Score,home_goals,away_goals,home_points,away_points
0,Fulham FC,Arsenal FC,0:3 (0:1),0,3,0,3
1,Crystal Palace,Southampton FC,1:0 (1:0),1,0,3,0
2,Liverpool FC,Leeds United,4:3 (3:2),4,3,3,0
3,West Ham United,Newcastle United,0:2 (0:0),0,2,0,3
4,West Bromwich Albion,Leicester City,0:3 (0:0),0,3,0,3
5,Tottenham Hotspur,Everton FC,0:1 (0:0),0,1,0,3
6,Sheffield United,Wolverhampton Wanderers,0:2 (0:2),0,2,0,3
7,Brighton & Hove Albion,Chelsea FC,1:3 (0:1),1,3,0,3
8,Burnley FC,Manchester United,0:1 (0:0),0,1,0,3
9,Manchester City,Aston Villa,2:0 (0:0),2,0,3,0


## Building Standing Table

We will `concat` again on the socres and calculate the standing table based on these scores.

* We will concatinate the following two tables
* First, we will calculate the games, points, and goals for the home teams
* Second, we will repeat the process for the away teams
* Next, we will sum the values from the two tables for each team
* Calculate the goal difference that is used as tie breaker in case of equal points
* Sort by the number of points and the goal difference

In [69]:
(
    pd
    .concat(
        [
            scores_table
            .rename(columns={'Home':'Team'})
            .groupby('Team')
            .agg(
                games = ('home_goals', 'count'), 
                goals = ('home_goals', 'sum'), 
                goals_against = ('away_goals', 'sum'), 
                points = ('home_points', 'sum')
            )
            ,
            scores_table
            .rename(columns={'Away':'Team'})
            .groupby('Team')
            .agg(
                games = ('away_goals', 'count'), 
                goals = ('away_goals', 'sum'), 
                goals_against = ('home_goals', 'sum'), 
                points = ('away_points', 'sum')
            )   
        ]
    )
    .groupby('Team')
    .sum()
    .assign(diff = lambda r : r.goals - r.goals_against)
    .sort_values(by=['points','diff'], ascending=False)
)

,games,goals,goals_against,points,diff
Team,,,,,
Everton FC,4,12,5,12,7
Aston Villa,4,11,4,9,7
Leicester City,4,12,7,9,5
Arsenal FC,4,8,5,9,3
Liverpool FC,4,11,11,9,0
Tottenham Hotspur,4,12,5,7,7
Chelsea FC,4,10,6,7,4
Leeds United,4,9,8,7,1
Manchester City,4,8,7,7,1
